In [1]:
# BLOK 1 
# Importovanie kniznic

import pandas as pd
import numpy as np
import json

from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Convolution1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, GRU, Concatenate
import tensorflow.keras.utils 
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from ast import literal_eval
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report

from tensorflow.keras.models import Sequential

C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
# BLOK 2 
# Nacitanie datasetu syntetickych kriviek

data_syn = pd.read_csv(r"C:\Users\nemci\OneDrive\Počítač\Bakalárska práca\Data\df_all.csv_v2")

In [3]:
# BLOK 3
# Diskretizacia

newDataAll = []
for row in data_syn['generic_bessell_v']:
    row = json.loads(row)
    minValueOfRow, maxValueOfRow = min(row), max(row)
    newRow = []
    for valueIndex in range(len(row)):
        row[valueIndex] = (row[valueIndex] - minValueOfRow) / (maxValueOfRow - minValueOfRow)
        newRow.append([row[valueIndex]])
    newDataAll.append(newRow)
newDataAll = np.array(newDataAll)

In [10]:
# BLOK 4
# Vytvorenie cieloveho atributu pre morfologicku klasifikaciu (0 = dotykove, 1 = oddelene) a cieloveho atributu pre klasifikaciu
# podla skvrnitosti (0 = neskvrnite, 1 = skrvnite). Zmena cielovych atributov na numpy pole

targetAll, targetSpotty = [], []

for index, row in data_syn.iterrows():
    if row["morphology"] == "over-contact" and row["spotty"] == 1:
        targetAll.append(0)
        targetSpotty.append(1)
    elif row["morphology"] == "over-contact" and row["spotty"] == 0:
        targetAll.append(0)
        targetSpotty.append(0)
    elif row["morphology"] == "detached" and row["spotty"] == 1:
        targetAll.append(1)
        targetSpotty.append(1)

    elif row["morphology"] == "detached" and row["spotty"] == 0:
        targetAll.append(1)
        targetSpotty.append(0)
        
targetAll = np.array(targetAll)
targetSpotty = np.array(targetSpotty)

print(len(targetAll))
print(len(targetSpotty))

990407
990407


In [11]:
# BLOK 6
# Rozdelenie upravenych kriviek na trenovaciu a testovaciu mnozinu v pomere 80:20

# Vsetky krivky
print("Number of light curves: " + str(len(newDataAll)) + "\nTarget attribute: " + str(len(targetAll)))
X_trainAll, X_testAll, y_trainAll, y_testAll = train_test_split(newDataAll, targetAll, test_size=0.2)
y_testAll = to_categorical(y_testAll, 2)
print("Length X_test: " + str(len(X_testAll)) + "\nLength y_test: " + str(len(y_testAll)))

# Oddelene krivky
X_trainDet, X_testDet, y_trainDet, y_testDet = train_test_split(newDataAll, targetSpotty, test_size = 0.2)
y_testDet = to_categorical(y_testDet, 2)
print("Length X_test: " + str(len(X_testDet)) + "\nLength y_test: " + str(len(y_testDet)))

# Dotykove krivky
X_trainOver, X_testOver, y_trainOver, y_testOver = train_test_split(newDataAll, targetSpotty, test_size = 0.2)
y_testOver = to_categorical(y_testOver, 2)
print("Length X_test: " + str(len(X_testOver)) + "\nLength y_test: " + str(len(y_testOver)))

Number of light curves: 990407
Target attribute: 990407
Length X_test: 198082
Length y_test: 198082
Length X_test: 198082
Length y_test: 198082
Length X_test: 198082
Length y_test: 198082


In [6]:
# BLOK 8
# Nacitanie modelu. Predikovanie triedy vsetkych syntetickych kriviek a vyhodnotenie modelu na testovacej mnozine syntetickych 
# kriviek pomocou confusion matrix a vypocet presnosti, navratnosti a f1 skore

modelAll = load_model("new_modelLstmCnnAll.hdf5")
y_pred = modelAll.predict(X_testAll)
y_pred2 = []
for i in y_pred:
    maximum = np.argmax(i)
    y_pred2.append(maximum) 
cm = confusion_matrix(y_testAll.argmax(axis=1), y_pred2)
print("Confusion Matrix (hierarchical model): \n" + str(cm))
rep = classification_report(y_testAll.argmax(axis=1), y_pred2, zero_division = 1)
print(str(rep))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Confusion Matrix (hierarchical model): 
[[ 18211      5]
 [    16 179850]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18216
           1       1.00      1.00      1.00    179866

    accuracy                           1.00    198082
   macro avg       1.00      1.00      1.00    198082
weighted avg       1.00      1.00      1.00    198082



In [12]:
# BLOK 9
# Nacitanie modelu. Predikovanie skvrnitosti oddelenych syntetickych kriviek a vyhodnotenie modelu na testovacej mnozine 
# syntetickych kriviek pomocou confusion matrix a vypocet presnosti, navratnosti a f1 skore

modelDet = load_model("new_modelDet.hdf5")
y_pred = modelDet.predict(X_testDet)
y_pred2 = []
for i in y_pred:
    maximum = np.argmax(i)
    y_pred2.append(maximum) 
cmD = confusion_matrix(y_testDet.argmax(axis=1), y_pred2)
print("Confusion Matrix (detached spotty/no spotty): \n" + str(cmD))
repDet = classification_report(y_testDet.argmax(axis=1), y_pred2, zero_division = 1)
print(str(repDet))

Confusion Matrix (detached spotty/no spotty): 
[[98442    65]
 [13005 86570]]
              precision    recall  f1-score   support

           0       0.88      1.00      0.94     98507
           1       1.00      0.87      0.93     99575

    accuracy                           0.93    198082
   macro avg       0.94      0.93      0.93    198082
weighted avg       0.94      0.93      0.93    198082



In [13]:
# BLOK 10
# Nacitanie modelu. Predikovanie skvrnitosti dotykovych syntetickych kriviek a vyhodnotenie modelu na testovacej mnozine 
# syntetickych kriviek pomocou confusion matrix a vypocet presnosti, navratnosti a f1 skore

modelOver = load_model("new_modelOver.hdf5")
y_pred = modelOver.predict(X_testOver)
y_pred2 = []
for i in y_pred:
    maximum = np.argmax(i)
    y_pred2.append(maximum) 
cmO = confusion_matrix(y_testOver.argmax(axis=1), y_pred2)
print("Confusion Matrix (overcontact spotty/no spotty): \n" + str(cmO))
repOver = classification_report(y_testOver.argmax(axis=1), y_pred2, zero_division = 1)
print(str(repOver))

Confusion Matrix (overcontact spotty/no spotty): 
[[54428 43982]
 [20053 79619]]
              precision    recall  f1-score   support

           0       0.73      0.55      0.63     98410
           1       0.64      0.80      0.71     99672

    accuracy                           0.68    198082
   macro avg       0.69      0.68      0.67    198082
weighted avg       0.69      0.68      0.67    198082

